In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import casadi as ca

In [2]:
from helper_fns import spawn_models
models = spawn_models('config/franka.yaml', 'config/attrs.yaml', 'config/contact.yaml', sym_vars = [])
rob = models['plane']

Building robot model from config/franka.urdf with TCP fr3_link8
Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['contact_1']
Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['contact_1', 'contact_2']


In [ ]:
from observer import EKF
rob.build_disc_dyn(0.002)
A, C, n, y = rob.linearized(np.zeros(rob.nx), np.zeros(7), rob.inv_mass_fn(np.zeros(7)))
ekf = EKF(rob)
%timeit ekf.step(0.01*np.ones(7), np.zeros(7), np.eye(7))
# 3x speedup from JITting the EKF step
for _ in range(100):
    ekf.step(0.01*np.ones(7), np.zeros(7))
print(ekf.x['mu'])
print(ekf.get_ext_state())

In [19]:
from impedance_controller import ImpedanceController
from robot import Robot
imp = ImpedanceController(pars={'imp_stiff':[300, 300, 300], 'imp_rest':[0, 0, 0.5]}, sym_vars = [])
rob = Robot('config/franka.urdf', subsys = [imp])
step = rob.build_disc_dyn(0.1)
# test that it converges as expected 
q = np.zeros(7)
dq = np.zeros(7)
tau = np.zeros(7)
state = {'q':q, 'dq':dq, 'tau_input':tau}
#%timeit step(**state)

for _ in range(100):
    res = step(**state)
    state['q'] = res['q_next']
    state['dq'] = res['dq_next']
    res_dict = rob.get_ext_state(state)
    print(res_dict['F_imp'])

#print(rob.inv_mass_fn(np.zeros(7)))


Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['imp_ctrl']
[-28.827, -0.00651662, -128.503]
[-26.3104, 0.00935689, -127.694]
[-22.0932, 0.0179492, -126.741]
[-17.3069, 0.0193694, -125.735]
[-12.4771, 0.0170414, -124.643]
[-7.86807, 0.0133725, -123.452]
[-3.60337, 0.00963115, -122.166]
[0.273173, 0.00636678, -120.797]
[3.76208, 0.00374294, -119.354]
[6.88673, 0.00173975, -117.843]
[9.68061, 0.000263911, -116.267]
[12.1799, -0.000795868, -114.623]
[14.4195, -0.0015437, -112.908]
[16.4307, -0.00206696, -111.116]
[18.2409, -0.00243434, -109.239]
[19.8726, -0.00269725, -107.271]
[21.3441, -0.00289245, -105.205]
[22.67, -0.00304483, -103.036]
[23.8611, -0.00317003, -100.759]
[24.9254, -0.00327671, -98.3703]
[25.8684, -0.00336849, -95.8699]
[26.6937, -0.00344563, -93.2585]
[27.4034, -0.00350643, -90.5392]
[27.9988, -0.00354841, -87.7175]
[28.4806, -0.0035693, -84.8008]
[28.8494, -0.00356776, -81.799]
[29.1063, -0.00354388, -78.7237]
[29.2526, -0.00349942, -75.5